# 03. Traffic Sign Classification - 결과 시각화

학습된 모델을 사용하여 테스트 이미지의 예측 결과를 시각화합니다.

## 시각화 과정
1. 환경 설정 및 라이브러리 설치
2. Google Drive 마운트 및 모델 로드
3. 테스트 데이터셋 준비
4. 예측 결과 시각화 (각 클래스당 4장씩)

## 1. 환경 설정

In [ ]:
# 필요한 라이브러리 설치
!pip install transformers pillow torch matplotlib -q

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from PIL import Image
from pathlib import Path
from torch.utils.data import Dataset as TorchDataset
from transformers import AutoImageProcessor, AutoModelForImageClassification

print("✅ 라이브러리 로드 완료")

## 2. Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. 작업 디렉토리 및 모델 로드

**중요**: `WORK_DIR`을 본인의 Google Drive 경로로 수정하세요.

In [ ]:
# Google Drive 내 작업 디렉토리 경로 설정
WORK_DIR = '/content/drive/MyDrive/2026_AI_Advanced_Study-main/3차시/02_Traffic_Sign_Classification'

# 작업 디렉토리로 이동
os.chdir(WORK_DIR)
print(f"현재 작업 디렉토리: {os.getcwd()}")

# 학습된 모델 경로 (고정)
MODEL_PATH = 'runs/classification/final_model'

# 모델 및 프로세서 로드
print(f"\n🤖 모델 로드 중...")
model = AutoModelForImageClassification.from_pretrained(MODEL_PATH)
processor = AutoImageProcessor.from_pretrained(MODEL_PATH)

model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"✅ 모델 로드 완료 (Device: {device})")

## 4. 커스텀 데이터셋 클래스 정의

In [ ]:
class TrafficSignDataset(TorchDataset):
    """로컬 이미지 파일을 로드하는 커스텀 데이터셋"""
    
    def __init__(self, data_dir, processor=None):
        self.data_dir = Path(data_dir)
        self.processor = processor
        self.samples = []
        
        # 클래스별 이미지 수집
        for class_idx in range(5):
            class_dir = self.data_dir / f'class_{class_idx}'
            if class_dir.exists():
                for img_path in sorted(class_dir.glob('*.jpg')):
                    self.samples.append((img_path, class_idx))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        return {'image': image, 'label': label}

print("✅ 커스텀 데이터셋 클래스 정의 완료")

## 5. 테스트 데이터 로드

In [ ]:
print("📥 테스트 데이터셋 로드 중...")

# 테스트 데이터셋 생성
test_dataset = TrafficSignDataset('data/images/test', processor=None)

print(f"✅ 테스트 데이터 준비 완료: {len(test_dataset)}장")

## 6. 예측 결과 시각화

각 클래스에서 4장씩 선택하여 총 20장의 예측 결과를 시각화합니다.
- 녹색 제목: 예측 성공 ✅
- 빨간색 제목: 예측 실패 ❌

In [ ]:
# 결과 저장 디렉토리 생성
os.makedirs('runs/classification_pred', exist_ok=True)

# 클래스별로 이미지 분류
print("🔍 클래스별 이미지 분류 중...")
samples_by_class = {i: [] for i in range(5)}
for idx in range(len(test_dataset)):
    item = test_dataset[idx]
    label = item['label']
    samples_by_class[label].append(idx)

# 각 클래스에서 4장씩 선택 (총 20장)
selected_indices = []
for class_idx in range(5):
    class_samples = samples_by_class[class_idx]
    num_to_select = min(4, len(class_samples))
    selected_indices.extend(class_samples[:num_to_select])

print(f"📊 시각화할 이미지: {len(selected_indices)}장 (각 클래스당 4장)\n")

# 5×4 그리드로 시각화
fig, axes = plt.subplots(5, 4, figsize=(16, 20))

for plot_idx, sample_idx in enumerate(selected_indices):
    row = plot_idx // 4
    col = plot_idx % 4
    
    item = test_dataset[sample_idx]
    image = item['image']
    true_label = item['label']
    
    # 전처리 및 예측
    inputs = processor(images=image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        pred_label = torch.argmax(outputs.logits, dim=-1).cpu().item()
        confidence = torch.softmax(outputs.logits, dim=-1)[0][pred_label].cpu().item()
    
    # 이미지 표시
    axes[row, col].imshow(image)
    axes[row, col].axis('off')
    
    # 제목 색상 (맞으면 녹색, 틀리면 빨간색)
    color = 'green' if pred_label == true_label else 'red'
    title = f"True: Class {true_label}\nPred: Class {pred_label}\nConf: {confidence:.2f}"
    axes[row, col].set_title(title, fontsize=10, color=color, fontweight='bold')

plt.tight_layout()
plt.savefig('runs/classification_pred/predictions.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ 예측 결과 시각화 완료: runs/classification_pred/predictions.png")
print(f"   각 클래스당 4장씩, 총 {len(selected_indices)}장 시각화")